# Depth One Search

This notebook illustrates depth one search for logical operators using the embedded code technique. The first version of the algorithm allows users to search for a depth-one level $t$ implementation of logical CP using multi-qubit gates. 

WARNING: this is an exhaustive method which only works for small codes of up to 30 qubits.
    
**Input:** 

- The X-checks $S_X$ and X-logicals $L_X$ of a CSS code;
- The desired level of the Clifford hierarchy $t$.
    
**Output:** 

- A depth-one implementation of a level $t$ logical operator using multi-qubit controlled phase gates, or FALSE if this is not possible

Users can run the algorithm on a variety of codes, including:
- 2D Toric code - depth one S_0 S3_1 operator
- Bring's Code (a 30-qubit LDPC code) - depth one S3[1][4][6] S[0][5][7] CZ[0,2][1,3][2,5][2,7][3,4][3,6], known partition
- Morphed Codes
- Symmetric Hypergraph product codes - depth one operators, known partitions

## Qubit Partitions - Alternative Algorithm Mode

The algorithm can also be run using a partition of the qubits (cList) - this is usful when testing for a known symmetry of the CSS code.

Users can test qubit partitions by uncommenting the cList variables below.

In [3]:
from add_parent_dir import *
from common import *
from NSpace import *
from clifford_LO import *
import itertools as iter

## default values
SX,LX,SZ,LZ = None,None,None,None
cList = None
t=2

## 2D toric Code
d = 3
SX, SZ, LX, LZ = toric2D(d)
# cList = toric2DPartition(d)

## Vasmer and Kubica [[5,1,2]] code
## logical S: S_1 S_2^3 S_3 CZ_{45}
# SX = '11010,01101'
# SZ = '11001,01110'
# cList = [[0],[1],[2],[3,4]]


## Vasmer and Kubica [[10,1,2]] code
## logical T: T[0-6] CCZ[7-9]
# SX = '1101100100,0111010010,0001111001'
# SZ = '1101100000,0001100010,0111010000,0101000001,0001111000,0001010100'
# t=3
# cList = [[0],[1],[2],[3],[4],[5],[6],[7,8,9]]

## Vasmer 3D Toric code from https://arxiv.org/abs/1801.04255 
# SXx = [
# [[5,6,7,8,9,10,11,12],[1,3,5],[2,4,7]],
# [[1,5,6,9],[2,6,7,10],[3,5,8,11],[4,7,8,12]],
# [[1,2,3,4,5,6,7,8],[6,9,10],[8,11,12]]
# ]
# LXx = [
# [3,4,8,11,12],
# [1,2,3,4],
# [1,3,5,9,11]
# ]

# n = 12
# SX = [] 
# LX = []
# for i in range(3):
#     temp = [ZMat(x) + i * n - 1 for x in SXx[i]]
#     SX.extend([set2Bin(3*n,x) for x in temp])
#     x = LXx[i]
#     x = ZMat(x) + i * n - 1
#     LX.append(set2Bin(3*n,x))
# SX = ZMat(SX)
# LX = ZMat(LX)
# t=3
# cList = [[i, i + n, i + 2*n] for i in range(n)]

## Bring's code from Fold-Transversal Gates for Quantum Codes
# SXx = [[17,24,9,7,29],
# [9,1,15,3,5],
# [7,1,12,2,4],
# [15,12,23,25,28],
# [2,16,10,6,23],
# [4,29,11,18,16],
# [3,28,8,13,19],
# [25,6,26,21,8],
# [10,18,22,30,26],
# [24,5,19,20,14],
# [20,13,21,30,27],
# [17,14,27,22,11]]
# LXx = [[1,6,11],[2,8,22],[4,5,25],[10,13,17],[24,28,30],[12,19,26],[9,16,27],[15,20,29]]
# n = 31
# ## convert to binary vectors, eliminate first col (above start from index 1)
# SX = ZMat([set2Bin(n,x) for x in SXx])[:,1:]
# LX = ZMat([set2Bin(n,x) for x in LXx])[:,1:]

# # tau_0
# cList = [[1],[2],[5],[10],[30],[20],[3,4],[6,7],[8,12],[9,11],[13,18],[14,15],[16,17],[19,25],[21,24],[22,23],[26,28],[27,29]]
# cList = [list(ZMat(c)- 1) for c in cList]

## Block code from Fold-Transversal Gates for Quantum Codes
# H = '111000,110100,110010,110001,101000,100100,100010,100000,001000,000100,000010,000001,011000,010100,010010,010001'
# SX = str2ZMatdelim(H).T
# LX = '1100110000000000,1100000000001100,0110011000000000,0110000000000110'
# LX = str2ZMatdelim(LX)


####################################################################
## SHPC Codes
####################################################################


## use pattern in the SHPC paper
# T = genH(3)

# # [[98,32,3]]
# T = '''1110100
# 1011010
# 0111001'''

# # # [[98,18,4]]
# T = '''1101000
# 1010100
# 0110010
# 1110001'''

# SX, SZ, LX, LZ = SHPC(T)
# cList = SHPC_partition(T)

startTimer()

Eq, SX,LX,SZ,LZ = CSSCode(SX,LX,SZ,LZ)

r,n = np.shape(SX)
k,n = np.shape(LX)

compact = n > 15
print('Original CSS Code')
print_SXLX(SX,LX,SZ,LZ,compact)
debug = False
print(f'Searching for level {t} logical operator.')
if cList is None: 
    print('Searching all possible qubit partitions.')
else:
    print(f'Searching with qubit partition {cList}.')
cList, target = depth_one_t(Eq,SX,LX,SZ,LZ,t,cList=cList,debug=debug)
print('elapsedTime',elapsedTime())
# if op is None and target is None:
#     cList, target = depth_one_t(Eq,SX,LX,SZ,LZ,t,debug=debug)
#     print('elapsedTime',elapsedTime())


Original CSS Code
SX
 X[0][6][9][11]
 X[1][7][10][11]
 X[2][8][9][10]
 X[3][6][12][14]
 X[4][7][13][14]
 X[5][8][12][13]
 X[0][3][15][17]
 X[1][4][16][17]
 X[2][5][15][16]
LX
 X[6][7][8]
 X[11][14][17]
SZ
 Z[0][2][9][15]
 Z[1][2][10][16]
 Z[0][1][11][17]
 Z[3][5][12][15]
 Z[4][5][13][16]
 Z[3][4][14][17]
 Z[6][8][9][12]
 Z[7][8][10][13]
 Z[6][7][11][14]
LZ
 Z[2][5][8]
 Z[15][16][17]
Searching for level 2 logical operator.
Searching all possible qubit partitions.

Depth-One Solution Found
Qubit Partition [[13, 17], [11, 12], [10, 15], [3, 7], [1, 5], [0, 8], [16], [14], [9], [6], [4], [2]]
Logical Operator:  S3[9][14][16] S[2][4][6] CZ[0,8][1,5][3,7][10,15][11,12][13,17]
Logical action:  S3[1] S[0]
Testing Action on Codewords
|00>L {0}
|10>L {2}
|01>L {6}
|11>L {0}
LO Test: True
elapsedTime 35.5
